# 1- Introdução

O objetivo deste manual é facilitar o consumo dos dados da API da Campanha Nacional de Vacinação contra a Covid-19 disponível no endereço https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao.

# 2- Utilização da API

API é um conjunto de rotinas e padrões de programação para acesso a um aplicativo de software ou plataforma baseado na Web. A sigla API refere-se ao termo em inglês "Application Programming Interface" que significa em tradução para o português "Interface de Programação de Aplicativos".

Através das APIs, os aplicativos podem se comunicar uns com os outros sem conhecimento ou intervenção dos usuários. Elas funcionam através da comunicação de diversos códigos, definindo comportamentos específicos de determinado objeto em uma interface.

# 2.1- Nomenclatura dos campos de notificação

Para consumir os dados disponibilizados pela API, utilizaremos a seguinte URL:

• https://imunizacao-es.saude.gov.br/_search

Com a seguinte credencial de acesso:

Usuário: imunizacao_public

Senha: qlto5t&7r_@+#Tlstigi

# 3- Configurando para consumir Informações da API

### Entrando com as identificações de usuário

In [1]:
# Chamando as bibliotecas necessárias

from requests import session
import pandas as pd

# Tamanho máximo de extração

MAX_SIZE = 10000

# Informações de URL, Login e Senha

USUARIO_API = ('imunizacao_public', 'qlto5t&7r_@+#Tlstigi')
BASE_URL = 'https://imunizacao-es.saude.gov.br/'

### Consultas de tamanho inferior ou igual a 10 mil

In [2]:
sessao = session()
res1 = sessao.post(BASE_URL+"_search", auth=USUARIO_API)
data = res1.json()
num_elementos = data['hits']['total']['value']

print(num_elementos)

10000


### Realizando filtros dentro da busca

In [3]:
if(num_elementos <= MAX_SIZE):
    
    query = {
        
"size": 10000,
        
          "query": {
        "bool": {
            "must": [
                { "terms": { "paciente_idade": ["12", "13", "14", "15", "16", "17"] } },
                {
                    "bool": {
                        "must": [
                            { "match_phrase": { "estabelecimento_uf": "AC" } }
                        ]
                    }
                }
            ]
        }
    }
}
   

response = sessao.post(BASE_URL+"_search", auth=USUARIO_API, json=query).json()

# Conferindo o número de elementos retornados pela API

print(len(response['hits']['hits']))

10000


### Consultas de tamanho superior a 10 mil

In [4]:
def scroll_function(scroll_id):
  scroll_query = {
      "scroll": "30m",
      "scroll_id": scroll_id
  }
  return sessao.post('https://imunizacao-es.saude.gov.br/_search/scroll', auth=USUARIO_API, json=scroll_query).json()

### Aplicando um filtro para refinamento da busca

In [22]:
    query = {
        
"size": 10000,
        
          "query": {
        "bool": {
            "must": [
                { "terms": { "paciente_idade": ["12", "13", "14", "15", "16", "17"] } },
                {
                    "bool": {
                        "must": [
                            { "match_phrase": { "estabelecimento_uf": "AC" } }
                        ]
                    }
                }
            ]
        }
    }
}

scroll_response = sessao.post(BASE_URL+"_search?scroll=30m", auth=USUARIO_API, json=query).json()
scroll_id = scroll_response['_scroll_id']

elements = scroll_response['hits']['hits']

# O scroll de fato

sequencia_scroll = scroll_function(scroll_id)

while(len(sequencia_scroll['hits']['hits']) != 0):
    
        try:
            
            scroll_id = sequencia_scroll['_scroll_id']
            elements = elements + sequencia_scroll['hits']['hits']
            sequencia_scroll = scroll_function(scroll_id)
        
        except:
            
            sequencia_scroll = scroll_function(scroll_id)
            print(len(elements))
            
    
print(f'Lista Total com {len(elements)}')

Lista Total com 148994


### Criando uma função que transforma o json em Dataframe

Devido ao fato das informações estarem dentro de um loop localizado na segunda chave 'hits', utilizou-se de um pequeno for para alocar os respectivos valores dentro de suas listas.

Obs.: Caso seja de interesse do usuário remover algumas colunas dentro do Dataframe, seria necessário apenas a exclusão das linhas / strings relacionadas aos nomes / rótulos presentes no comando abaixo.

In [6]:
def Dataframe_pandas(data):
    
    # Declarando as listas

    rotulos = ['vacina_categoria_codigo','estalecimento_noFantasia','paciente_endereco_coPais','vacina_lote'
          ,'document_id','paciente_endereco_nmPais','vacina_fabricante_referencia','vacina_nome'
          ,'estabelecimento_valor','timestamp','paciente_dataNascimento','estabelecimento_razaoSocial'
          ,'paciente_endereco_nmMunicipio','vacina_descricao_dose','vacina_dataAplicacao','paciente_nacionalidade_enumNacionalidade'
          ,'estabelecimento_uf','vacina_grupoAtendimento_codigo','vacina_grupoAtendimento_nome','vacina_fabricante_nome'
          ,'vacina_codigo','status','paciente_endereco_coIbgeMunicipio','id_sistema_origem' ,'sistema_origem' ,'paciente_endereco_uf'
          ,'paciente_idade','paciente_racaCor_valor','paciente_racaCor_codigo','paciente_enumSexoBiologico','data_importacao_rnds'
          ,'version','dt_deleted','paciente_id','estabelecimento_municipio_nome','paciente_endereco_cep','vacina_categoria_nome'
          ,'estabelecimento_municipio_codigo']

    for i in range(len(rotulos)):
         globals()[rotulos[i]] = []
        
    # Alocando os valores nas listas por meio de um loop
    
    for i in range(0,len(data)):
    
        vacina_categoria_codigo.append(data[i]['_source']['vacina_categoria_codigo'])
        estalecimento_noFantasia.append(data[i]['_source']['estalecimento_noFantasia'])
        paciente_endereco_coPais.append(data[i]['_source']['paciente_endereco_coPais'])
        vacina_lote.append(data[i]['_source']['vacina_lote'])
        document_id.append(data[i]['_source']['document_id'])
        vacina_fabricante_referencia.append(data[i]['_source']['vacina_fabricante_referencia'])
        vacina_nome.append(data[i]['_source']['vacina_nome'])
        estabelecimento_valor.append(data[i]['_source']['estabelecimento_valor'])
        timestamp.append(data[i]['_source']['@timestamp'])
        paciente_dataNascimento.append(data[i]['_source']['paciente_dataNascimento'])
        estabelecimento_razaoSocial.append(data[i]['_source']['estabelecimento_razaoSocial'])
        paciente_endereco_nmMunicipio.append(data[i]['_source']['paciente_endereco_nmMunicipio'])
        vacina_descricao_dose.append(data[i]['_source']['vacina_descricao_dose'])
        vacina_dataAplicacao.append(data[i]['_source']['vacina_dataAplicacao'])
        paciente_nacionalidade_enumNacionalidade.append(data[i]['_source']['paciente_nacionalidade_enumNacionalidade'])
        estabelecimento_uf.append(data[i]['_source']['estabelecimento_uf'])
        vacina_grupoAtendimento_codigo.append(data[i]['_source']['vacina_grupoAtendimento_codigo'])
        vacina_grupoAtendimento_nome.append(data[i]['_source']['vacina_grupoAtendimento_nome'])
        vacina_codigo.append(data[i]['_source']['vacina_codigo'])
        status.append(data[i]['_source']['status'])
        paciente_endereco_coIbgeMunicipio.append(data[i]['_source']['paciente_endereco_coIbgeMunicipio'])
        id_sistema_origem.append(data[i]['_source']['id_sistema_origem'])
        sistema_origem.append(data[i]['_source']['sistema_origem'])
        paciente_idade.append(data[i]['_source']['paciente_idade'])
        paciente_racaCor_valor.append(data[i]['_source']['paciente_racaCor_valor'])
        paciente_racaCor_codigo.append(data[i]['_source']['paciente_racaCor_codigo'])
        paciente_enumSexoBiologico.append(data[i]['_source']['paciente_enumSexoBiologico'])
        data_importacao_rnds.append(data[i]['_source']['data_importacao_rnds'])
        version.append(data[i]['_source']['@version'])
        dt_deleted.append(data[i]['_source']['dt_deleted'])
        estabelecimento_municipio_codigo.append(data[i]['_source']['estabelecimento_municipio_codigo'])
        paciente_id.append(data[i]['_source']['paciente_id'])
        estabelecimento_municipio_nome.append(data[i]['_source']['estabelecimento_municipio_nome'])
        paciente_endereco_cep.append(data[i]['_source']['paciente_endereco_cep'])
        vacina_categoria_nome.append(data[i]['_source']['vacina_categoria_nome'])
        
    # Criando-se um Dataframe para alocar os valores
    
    dados = pd.DataFrame(list(zip(vacina_categoria_codigo, estalecimento_noFantasia,paciente_endereco_coPais,
                                 vacina_lote,document_id,vacina_fabricante_referencia,estabelecimento_valor,
                                 timestamp,paciente_dataNascimento,estabelecimento_razaoSocial,paciente_endereco_nmMunicipio,
                                 vacina_descricao_dose,vacina_dataAplicacao,paciente_nacionalidade_enumNacionalidade,
                                 estabelecimento_uf,vacina_grupoAtendimento_codigo,vacina_grupoAtendimento_nome,
                                 vacina_codigo,status,paciente_endereco_coIbgeMunicipio,id_sistema_origem,
                                 sistema_origem,paciente_idade,paciente_racaCor_codigo,paciente_enumSexoBiologico,
                                 data_importacao_rnds,dt_deleted,estabelecimento_municipio_codigo,paciente_id,
                                 estabelecimento_municipio_nome,paciente_endereco_cep,vacina_categoria_nome)), 
                         
                         columns = ['vacina_categoria_codigo', 'estalecimento_noFantasia','paciente_endereco_coPais',
                                 'vacina_lote','document_id','vacina_fabricante_referencia','estabelecimento_valor',
                                 'timestamp','paciente_dataNascimento','estabelecimento_razaoSocial','paciente_endereco_nmMunicipio',
                                 'vacina_descricao_dose','vacina_dataAplicacao','paciente_nacionalidade_enumNacionalidade',
                                 'estabelecimento_uf','vacina_grupoAtendimento_codigo','vacina_grupoAtendimento_nome',
                                 'vacina_codigo','status','paciente_endereco_coIbgeMunicipio','id_sistema_origem',
                                 'sistema_origem','paciente_idade','paciente_racaCor_codigo','paciente_enumSexoBiologico',
                                 'data_importacao_rnds','dt_deleted','estabelecimento_municipio_codigo','paciente_id',
                                 'estabelecimento_municipio_nome','paciente_endereco_cep','vacina_categoria_nome'])

    return(dados)

### Visualizando os resultados obtidos - Extração inferior a 10 mil

In [ ]:
# Gerando o Dataframe para os dados menores que 10 mil observações

dados_menores_10_mil = response['hits']['hits']

df_menores_10_mil = Dataframe_pandas(data = dados_menores_10_mil)

df_menores_10_mil

### Visualizando os resultados obtidos - Extração superior a 10 mil

In [23]:
# Gerando o Dataframe para os dados maiores que 10 mil observações

dados_maiores_10_mil = elements

df_maiores_10_mil = Dataframe_pandas(data = dados_maiores_10_mil)

df_maiores_10_mil

,vacina_categoria_codigo,estalecimento_noFantasia,paciente_endereco_coPais,vacina_lote,document_id,vacina_fabricante_referencia,estabelecimento_valor,timestamp,paciente_dataNascimento,estabelecimento_razaoSocial,...,paciente_idade,paciente_racaCor_codigo,paciente_enumSexoBiologico,data_importacao_rnds,dt_deleted,estabelecimento_municipio_codigo,paciente_id,estabelecimento_municipio_nome,paciente_endereco_cep,vacina_categoria_nome
0,2,ESF TUFIC MIZAEL SAADY,10,210400,a3cb71c1-e77a-4267-957b-29d924e8346c-i0b0,Organization/152,2001349,2022-04-07T21:47:42.342Z,2005-03-03,PREFEITURA MUNICIPAL DE BRASILEIA,...,17,03,F,2022-04-07T12:49:17.000Z,None,120010,c6bed535fbb997220e7df8885ac9163cb73b233906755a...,BRASILEIA,69932,Faixa Etária
1,2,CENTRO DE SAUDE PAULO ALCIONE MARQUES,10,FP8073,a885aec4-9b94-41d1-9c0d-955557322df4-i0b0,Organization/28290,2001772,2022-04-08T09:16:29.475Z,2008-08-05,PREFEITURA MUNICIPAL DE SANTA ROSA DO PURUS,...,13,05,M,2022-04-07T19:07:46.000Z,None,120043,3412e071f6b5b7a7a686d123a85e5fa92c8f3f8389007b...,SANTA ROSA DO PURUS,None,Faixa Etária
2,2,DEPARTAMENTO DE VIGILANCIA EPIDEMIOLOGICA E AM...,10,FP8073,13e9afea-f430-4a19-b308-2e959ac440c9-i0b0,Organization/28290,6917291,2022-04-07T21:47:54.249Z,2010-03-09,PREFEITURA MUNICIPAL DE RIO BRANCO,...,12,04,F,2022-04-07T17:22:26.000Z,None,120040,b483cc6a3d901975a512423c453724c56af48c0d0eaf10...,RIO BRANCO,None,Faixa Etária
3,2,POSTO DE SAUDE MAURICIO PERES NOBRE,10,210381,04f06d3e-8ea7-4bf0-a4f6-a1357c0b6891-i0b0,Organization/152,7560583,2022-04-07T21:33:53.569Z,2005-08-02,PREFEITURA MUNICIPAL DE CRUZEIRO DO SUL,...,16,04,M,2022-04-07T15:44:33.000Z,None,120020,07bbb055b30f97f748e734eb6abc60add53683597bcb11...,CRUZEIRO DO SUL,69980,Faixa Etária
4,2,DEPARTAMENTO DE VIGILANCIA EPIDEMIOLOGICA E AM...,10,FP8073,ad600000-700d-4c6c-933a-6132eab86e47-i0b0,Organization/28290,6917291,2022-04-08T09:17:05.932Z,2008-11-14,PREFEITURA MUNICIPAL DE RIO BRANCO,...,13,03,M,2022-04-07T22:24:57.000Z,None,120040,71377f75a3c1c433b188e152657a36a6276710e30b02e1...,RIO BRANCO,None,Faixa Etária
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148989,2,UNIDADE BASICA DE SAUDE GEOVANA MARIA FREITAS ...,10,FF8842,00e96df4-1bcf-4b46-b9ad-3a104fdd38f5-i0b0,Organization/00394544000851,2000598,2022-02-05T10:45:16.354Z,2007-05-30,PREFEITURA MUNICIPAL DE CRUZEIRO DO SUL,...,14,04,M,2021-10-08T16:09:22.000Z,None,120020,d78a4f1d6c8d02efa46ec2d10c58286dfccdc9725439a1...,CRUZEIRO DO SUL,69980,Faixa Etária
148990,2,DEPARTAMENTO DE VIGILANCIA EPIDEMIOLOGICA E AM...,10,FF8848,5ffdbd61-26cb-4603-999f-db1ac873c579-i0b0,Organization/00394544000851,6917291,2022-02-05T10:45:35.204Z,2004-11-15,PREFEITURA MUNICIPAL DE RIO BRANCO,...,16,03,F,2021-10-08T15:21:41.000Z,None,120040,58066f9f0193257446dbb16bc803b0c742c597c615748a...,RIO BRANCO,69907,Faixa Etária
148991,2,DEPARTAMENTO DE VIGILANCIA EPIDEMIOLOGICA E AM...,10,FF8848,81321a6b-606a-47d2-9539-15277ed1a44a-i0b0,Organization/00394544000851,6917291,2022-02-05T10:54:18.180Z,2004-09-01,PREFEITURA MUNICIPAL DE RIO BRANCO,...,17,03,F,2021-10-09T12:34:42.000Z,None,120040,fd726f55c750317db013edf7addec25892f4ae7fde63f6...,RIO BRANCO,69921,Faixa Etária
148992,2,UNIDADE DE SAUDE DA FAMILIA ANTONIO RODRIGUES ...,10,FF8848,f94eebcc-8b16-4d0b-9df8-64cf54071df2-i0b0,Organization/00394544000851,5354072,2022-02-05T11:16:29.331Z,2008-12-20,PREFEITURA MUNICIPAL DE JORDAO,...,12,04,F,2021-10-11T14:54:01.000Z,None,120032,1f15fbfa5f9633b07c67c95151cdefd670959004e0a912...,JORDAO,69975,Faixa Etária


# Exportando a base de dados 

In [ ]:
df_menores_10_mil.to_csv("df_menores_10_mil.txt", index=False, encoding='utf-8-sig',sep =';')

df_maiores_10_mil.to_csv("df_maiores_10_mil.txt", index=False, encoding='utf-8-sig',sep =';')